In [1]:
#################################
# filenames
#################################

number = 27

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1.1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(30)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(30)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1.1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in url_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in url_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1.1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(30)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(30)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1.1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(30)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(30)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1.1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:100])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615326064.472187
500
1/500 16,1,6,5
histo1 2
2/500 1,2,66,5/6
histo1 1
3/500 57/6,6,47,664
histo1 1
4/500 66,57/3,5/6,542/6
histo1 1
5/500 6,3,37,6
histo1 1
6/500 b7,2,1,5
histo1 2
7/500 37,27,17,565
histo1 1
8/500 2,1,26,2
histo1 1
9/500 67,1,27,47
histo1 1
10/500 1,4,16,564
histo1 4
11/500 57/6,6,47,5
histo1 2
12/500 6,7/6,6,4
histo1 1
13/500 47,164,46,464
histo1 1
14/500 242,7,6,56/6
histo1 1
15/500 6,L7,2,66
histo1 2
16/500 5/6,1,164,1
histo1 1
17/500 b4,b47,C2,b6
histo1 1
18/500 1,56,67,b5
histo1 1
19/500 37,47,C67,17
histo1 1
20/500 57/2,27,265,36
histo1 1
21/500 3,L7,6,b6
histo1 2
22/500 6,66,2,6
histo1 1
23/500 6,5/5,6,1
histo1 2
24/500 57,6,Y4,1
histo1 1
25/500 37,5/2,5/5,57
histo1 1
26/500 56,6,2,56
histo1 2
27/500 6,5,6,L7
histo1 2
28/500 264,26,2,26
histo1 1
29/500 57/5,b7,6,5
histo1 1
30/500 464,67,264,66
histo1 1
31/500 b7,4,1,L27
histo1 1
32/500 56,5,1,56
histo1 1
33/500 66,5,6,b7
histo1 1
34/500 16,37,27,4
histo1 1
35/500 4,b46,5,56
histo1 1
36/500 3,6,5,56
histo1 1
37

histo1 1
290/500 4,2,464,1
histo1 1
291/500 6,57/5,6,4/7
histo1 1
292/500 16,b16,27,5
histo1 4
293/500 b4,1,17,57/4
histo1 1
294/500 1,5,47,2
histo1 1
295/500 464,1,56,4
histo1 2
296/500 564,564/6,6,1
histo1 3
297/500 27,b27,17,27
histo1 1
298/500 5,6,4,77
histo1 1
299/500 4,16,1,56
histo1 8
300/500 5,1,7,57/6
histo1 1
301/500 47,67,4,67
histo1 1
302/500 664,66,364,66
histo1 1
303/500 664,4,26,664
histo1 1
304/500 17,2,17,2
histo1 2
305/500 27,6,164,4
histo1 1
306/500 57,1,b3,5
histo1 1
307/500 5,67,47,27
histo1 2
308/500 5,6,264,364
histo1 1
309/500 6,1,6,36
histo1 2
310/500 1,56,5,67
histo1 1
311/500 5,3,1,4
histo1 3
312/500 1,2,4,5
histo1 29
313/500 665,5,b1,D165
histo1 1
314/500 6,27,67,57/6
histo1 1
315/500 16,1,564,5
histo1 1
316/500 164,47,664,5/2
histo1 2
317/500 b6,1,5/6,564/6
histo1 1
318/500 2,4,464,2
histo1 1
319/500 2,1,37,27
histo1 1
320/500 1,67,47,1
histo1 2
321/500 6,4,442,1
histo1 1
322/500 164,47,165,67
histo1 1
323/500 4,1,27,36
histo1 1
324/500 4,47,4,37
histo1 2
3

In [ ]:
##########################
# list errors here
##########################
182/500 6,66,6,36
2. Unable to extract for 6 66 6 36 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [2]:
##########################
# test errors here
##########################
mykey = '6,66,6,36'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [3]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 1
# print(histo[:20])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 372, 53, 18, 12, 7, 6, 2, 3, 2, 1, 3, 2, 1, 1, 0, 1, 1, 1, 1]
